In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import json

data = pd.read_csv('train.csv')
data.head()


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.10,0.90,7,286,4,0,1,0,sales,low
1,0.89,0.93,4,249,3,0,0,0,sales,low
2,0.38,0.50,2,132,3,0,1,0,accounting,low
3,0.95,0.71,4,151,4,0,0,0,sales,medium
4,0.84,0.84,5,163,3,0,0,0,technical,low


In [2]:
print data.shape

(11238, 10)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    data,
    data.left,
    test_size=0.2,
    random_state=0)

print len(X_train)

8990


In [4]:
map_range_sl = lambda x : '<=0.37' if x <= 0.37 else ('>0.37 and <=0.51' if x <= 0.51 else ('>0.51 and <=0.64' if x<=0.64 else ('>0.64 and <=0.76' if x<=0.76 else ('>0.76 and <=0.87' if x<=0.87 else '>0.87'))))
X_train.satisfaction_level = X_train.satisfaction_level.apply(map_range_sl)
X_test.satisfaction_level = X_test.satisfaction_level.apply(map_range_sl)
map_range_le = lambda x : '<=0.52' if x <= 0.52 else ('>0.52 and <=0.61' if x <= 0.61 else ('>0.61 and <=0.72' if x<=0.72 else ('>0.72 and <=0.83' if x<=0.83 else ('>0.83 and <=0.91' if x<=0.91 else '>0.91'))))
X_train.last_evaluation = X_train.last_evaluation.apply(map_range_le)
X_test.last_evaluation = X_test.last_evaluation.apply(map_range_le)
map_range_np = lambda x : '<=3' if x <=3 else ('>3 and <=6' if x<=6 else '>6')
X_train.number_project = X_train.number_project.apply(map_range_np)
X_test.number_project = X_test.number_project.apply(map_range_np)
map_range_amh = lambda x : '<=156' if x <=156 else ('>156 and <=199' if x<=199 else ('>199 and <=245' if x<=245 else '>245'))
X_train.average_montly_hours = X_train.average_montly_hours.apply(map_range_amh)
X_test.average_montly_hours = X_test.average_montly_hours.apply(map_range_amh)
map_range_ts = lambda x : '<=3' if x <=3 else ('>3 and <=6' if x<=6 else '>6')
X_train.time_spend_company = X_train.time_spend_company.apply(map_range_ts)
X_test.time_spend_company = X_test.time_spend_company.apply(map_range_ts)

/home/tarunm/.local/lib/python2.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
# X_train_sorted = X_train.sort_values(['satisfaction_level'])
# # X_train_sorted.satisfaction_level
# # X_train_sorted.loc[data[max_col]==cond]
# # X_train_sorted.iloc[[7490]]
# len(X_train_sorted.loc[X_train_sorted['satisfaction_level']<1]) - len(X_train_sorted.loc[X_train_sorted['satisfaction_level']<0.87])
# #0.37
# #0.51
# #0.64
# #0.76
# #0.87

In [6]:
X_train.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
1994,>0.76 and <=0.87,>0.61 and <=0.72,>3 and <=6,>245,<=3,0,0,0,sales,low
2032,>0.64 and <=0.76,>0.52 and <=0.61,>3 and <=6,>156 and <=199,<=3,0,0,0,hr,low
883,>0.51 and <=0.64,<=0.52,<=3,<=156,>3 and <=6,0,0,0,support,low
7069,>0.37 and <=0.51,>0.91,<=3,>156 and <=199,>3 and <=6,1,0,0,IT,low
10426,<=0.37,>0.72 and <=0.83,<=3,>156 and <=199,>3 and <=6,0,0,0,IT,low


In [7]:
def create_tree_node(value, is_leaf):
    new_node = dict()
    new_node['value'] = value
    new_node['is_leaf'] = is_leaf
    new_node['children'] = dict()
    return new_node

In [8]:
def display_tree(t,index):
    print t
    tabs = ''
    for i in range(0,index):
        tabs+='\t'
    print tabs+str(t['value'])
    for c in t['children']:
        i = (index+1)
        display_tree(c,i)

In [9]:
def print_tree(curr_data,curr_parent_object,curr_condition, exclude_cols,decision_tree):
    
    curr_parent_str = curr_parent_object['value']
    if curr_parent_str not in exclude_cols:
        exclude_cols.append(curr_parent_str)
    #print 'Current Parent object : '+str(curr_parent_object)
    #print 'Current Condition : '+str(curr_condition)
    m = len(curr_data)
    G = dict()
    for col in curr_data:
        if col in exclude_cols:
            continue
        I = 0.0
        col_pos = 0
        col_neg = 0
        for categ in curr_data[col].unique():
            #print 'col :: '+col+' ; categ :: '+str(categ)
            #print curr_data.groupby([col])['left'].value_counts()[categ]
            pos = 0
            neg = 0
            if 1 in curr_data.groupby([col])['left'].value_counts()[categ]: 
                pos = curr_data.groupby([col])['left'].value_counts()[categ][1]
            if 0 in curr_data.groupby([col])['left'].value_counts()[categ]:
                neg = curr_data.groupby([col])['left'].value_counts()[categ][0]
            col_pos += pos
            col_neg += neg
            pos_ratio = float(pos)/float(pos+neg)
            neg_ratio = float(neg)/float(pos+neg)
            if pos_ratio == 0 or pos_ratio == 1:
                entropy = 0
            else:
                entropy = -((pos_ratio*np.log(pos_ratio)) + (neg_ratio*np.log(neg_ratio)))
            I += (float(pos+neg)/float(m))*entropy
        col_pos_ratio = float(col_pos)/float(col_pos+col_neg)
        col_neg_ratio = float(col_neg)/float(col_pos+col_neg)
        if col_pos_ratio == 0 or col_pos_ratio == 1:
            E = 0
        else:
            E = -((col_pos_ratio*np.log(col_pos_ratio)) + (col_neg_ratio*np.log(col_neg_ratio)))
        G[col] = (E-I)
    if not G:
        if curr_parent_str == 'dummy_parent':
            return decision_tree
        for categ in curr_data[col].unique():
            if 0 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition] and 1 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                if curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition][0] > curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition][1]:
                    #print 'Left'
                    new_node = create_tree_node('0',True)
                    curr_parent_object['children'][curr_condition] = new_node
                else:
                    new_node = create_tree_node('1',True)
                    curr_parent_object['children'][curr_condition] = new_node
            elif 0 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                new_node = create_tree_node('0',True)
                curr_parent_object['children'][curr_condition] = new_node
            elif 1 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                new_node = create_tree_node('1',True)
                curr_parent_object['children'][curr_condition] = new_node
            else:
                new_node = create_tree_node('0',True)
                curr_parent_object['children'][curr_condition] = new_node
        return decision_tree
    
    max_col = max(G, key=G.get)
    
    new_node = create_tree_node(max_col,False)
    if decision_tree is None:
        decision_tree = new_node
    else:
        curr_parent_object['children'][curr_condition] = new_node
    
    res_list = list()
    next_conds = list()
    for cond in curr_data[max_col].unique():
        if 0 in curr_data.groupby([max_col])['left'].value_counts()[cond]:
            res_list.append(curr_data.groupby([max_col])['left'].value_counts()[cond][0])
        else:
            res_list.append(0)
        if 1 in curr_data.groupby([max_col])['left'].value_counts()[cond]:
            res_list.append(curr_data.groupby([max_col])['left'].value_counts()[cond][1])
        else:
            res_list.append(0)
        res_sum = sum(res_list)
        if res_sum == 0:
            new_leaf_node = create_tree_node('0',True)
            new_node['children'][cond] = new_leaf_node
        else:
            ratio = max(res_list)/res_sum
            if ratio > 0.90:
                if res_list[0] == max(res_list):
                    new_leaf_node = create_tree_node('0',True)
                    new_node['children'][cond] = new_leaf_node
                else:
                    new_leaf_node = create_tree_node('1',True)
                    new_node['children'][cond] = new_leaf_node
            else:
                next_conds.append(cond)
    
    
    
    temp_exclude_cols = exclude_cols[:]
    for cond in next_conds:
        print_tree(curr_data.loc[curr_data[max_col]==cond].copy() , new_node, cond,temp_exclude_cols,decision_tree)
    
    return decision_tree

In [10]:
decision_tree = None
dummy_node = dict()
dummy_node['value'] = 'dummy_parent'
decision_tree = print_tree(X_train.copy(),dummy_node,'dummy_condition',['left'],decision_tree)
print json.dumps(decision_tree)

{"is_leaf": false, "children": {">0.64 and <=0.76": {"is_leaf": false, "children": {"<=3": {"is_leaf": false, "children": {">0.72 and <=0.83": {"is_leaf": false, "children": {"management": {"is_leaf": false, "children": {"high": {"is_leaf": true, "children": {}, "value": "0"}, "medium": {"is_leaf": false, "children": {"0": {"is_leaf": true, "children": {}, "value": "0"}}, "value": "promotion_last_5years"}, "low": {"is_leaf": false, "children": {"0": {"is_leaf": true, "children": {}, "value": "0"}}, "value": "promotion_last_5years"}}, "value": "salary"}, "product_mng": {"is_leaf": false, "children": {"high": {"is_leaf": false, "children": {"0": {"is_leaf": true, "children": {}, "value": "0"}}, "value": "promotion_last_5years"}, "medium": {"is_leaf": false, "children": {"0": {"is_leaf": true, "children": {}, "value": "0"}}, "value": "promotion_last_5years"}, "low": {"is_leaf": true, "children": {}, "value": "0"}}, "value": "salary"}, "technical": {"is_leaf": true, "children": {}, "value"

In [20]:
def validate_row(row,tree_node):
    if tree_node['is_leaf']:
        if row['left'] == 1 and tree_node['value'] == '1':
            #true positive
            return 2
        elif row['left'] == 1 and tree_node['value'] == '0':
            #false negative
            return -1
        elif row['left'] == 0 and tree_node['value'] == '1':
            #false positive
            return 0
        elif row['left'] == 0 and tree_node['value'] == '0':
            #true negative
            return 1
    key = tree_node['value']
    val = row[key]
    if val not in tree_node['children']:
        #error considered as true negative
        return 1
    else:
        return validate_row(row,tree_node['children'][val])

In [26]:
m_train = len(X_train)
hits_train = 0
tp = 0
tn = 0
fp = 0
fn = 0
for index,row in X_train.iterrows():
    verdict = validate_row(row,decision_tree)
    if verdict == -1:
        fn += 1
    elif verdict == 0:
        fp += 1
    elif verdict == 1:
        hits_train += 1
        tn += 1
    elif verdict == 2:
        hits_train += 1
        tp += 1
    
# print 'Total : '+str(m_train)
# print 'Hits : '+str(hits_train)
accuracy = float(hits_train)/float(m_train)
misclassification = 1.0 - accuracy
print 'Train Accuracy : '+str(accuracy*100)
print 'Misclassification error : '+str(misclassification * 100)
precision = float(tp)/(float(tp)+float(fp))
recall = float(tp)/(float(tp)+float(fn))
print 'Precision : '+str(precision)
print 'Recall : '+str(recall)

f1_score = 2.0/((1.0/recall)+(1.0/precision))
print 'F1 score : '+str(f1_score)

Train Accuracy : 97.6529477197
Misclassification error : 2.34705228031
Precision : 0.919047619048
Recall : 0.988821611551
F1 score : 0.952658739062


In [27]:
m_test = len(X_test)
hits_test = 0
tp = 0
tn = 0
fp = 0
fn = 0
for index,row in X_test.iterrows():
    verdict = validate_row(row,decision_tree)
    if verdict == -1:
        fn += 1
    elif verdict == 0:
        fp += 1
    elif verdict == 1:
        hits_test += 1
        tn += 1
    elif verdict == 2:
        hits_test += 1
        tp += 1
    
# print 'Total : '+str(m_train)
# print 'Hits : '+str(hits_train)
accuracy = float(hits_test)/float(m_test)
misclassification = 1.0 - accuracy
print 'Test Accuracy : '+str(accuracy*100)
print 'Misclassification error : '+str(misclassification * 100)
precision = float(tp)/(float(tp)+float(fp))
recall = float(tp)/(float(tp)+float(fn))
print 'Precision : '+str(precision)
print 'Recall : '+str(recall)
f1_score = 2.0/((1.0/recall)+(1.0/precision))
print 'F1 score : '+str(f1_score)

Test Accuracy : 94.5729537367
Misclassification error : 5.42704626335
Precision : 0.844247787611
Recall : 0.933463796477
F1 score : 0.886617100372


In [ ]:
# sl_dict = {k:i for i, k in enumerate(X_train.satisfaction_level.unique(), 0)}
# le_dict = {k:i for i, k in enumerate(X_train.last_evaluation.unique(), 0)}
# np_dict = {k:i for i, k in enumerate(X_train.number_project.unique(), 0)}
# amh_dict = {k:i for i, k in enumerate(X_train.average_montly_hours.unique(), 0)}
# tsc_dict = {k:i for i, k in enumerate(X_train.time_spend_company.unique(), 0)}
# sales_dict = {k:i for i, k in enumerate(X_train.sales.unique(), 0)}
# salary_dict = {k:i for i, k in enumerate(X_train.salary.unique(), 0)}

# X_train.loc[:, 'satisfaction_level'] = X_train.loc[:, 'satisfaction_level'].map(sl_dict)
# X_test.loc[:, 'satisfaction_level'] = X_test.loc[:, 'satisfaction_level'].map(sl_dict)

# X_train.loc[:, 'last_evaluation'] = X_train.loc[:, 'last_evaluation'].map(le_dict)
# X_test.loc[:, 'last_evaluation'] = X_test.loc[:, 'last_evaluation'].map(le_dict)

# X_train.loc[:, 'number_project'] = X_train.loc[:, 'number_project'].map(np_dict)
# X_test.loc[:, 'number_project'] = X_test.loc[:, 'number_project'].map(np_dict)

# X_train.loc[:, 'average_montly_hours'] = X_train.loc[:, 'average_montly_hours'].map(amh_dict)
# X_test.loc[:, 'average_montly_hours'] = X_test.loc[:, 'average_montly_hours'].map(amh_dict)

# X_train.loc[:, 'time_spend_company'] = X_train.loc[:, 'time_spend_company'].map(tsc_dict)
# X_test.loc[:, 'time_spend_company'] = X_test.loc[:, 'time_spend_company'].map(tsc_dict)

# X_train.loc[:, 'sales'] = X_train.loc[:, 'sales'].map(sales_dict)
# X_test.loc[:, 'sales'] = X_test.loc[:, 'sales'].map(sales_dict)

# X_train.loc[:, 'salary'] = X_train.loc[:, 'salary'].map(salary_dict)
# X_test.loc[:, 'salary'] = X_test.loc[:, 'salary'].map(salary_dict)

# # print X_train.head()

# clf = DecisionTreeClassifier()
# clf.fit(X_train,y_train)
# print clf.score(X_test,y_test)

In [ ]:
# data.loc[data['salary']=='low'].head()
# data['salary'].unique()
